In [2]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [14]:
X,y = make_classification(n_samples=10000,n_features= 10,n_informative=3)

In [22]:
X.shape,y.shape

((10000, 10), (10000,))

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)


In [24]:
X_train

array([[ 0.54582212, -0.17659407, -2.27436819, ..., -1.53638955,
         1.06254499,  0.91367063],
       [ 0.01562341,  0.53017217, -3.23586588, ..., -0.7256707 ,
         1.75295339, -0.1755643 ],
       [-0.12365858, -1.54349423,  0.23060957, ...,  0.22033538,
         0.5200874 , -0.67180151],
       ...,
       [-0.44497183, -0.50282655, -1.1652278 , ..., -0.25377939,
        -0.34020279, -1.77051673],
       [ 0.39516968,  0.82888878, -0.17812382, ..., -1.40044718,
         0.37477251,  0.85700586],
       [ 0.41655297, -0.57508754, -0.21385992, ..., -1.7380077 ,
         2.29715378,  1.30220542]])

In [9]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))


Decision Tree accuracy 0.9065


## Bagging

In [12]:
bag = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500,
    max_samples = 0.5,
    bootstrap = True,
    random_state = 42
)

In [13]:
bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [14]:
y_predict = bag.predict(X_test)

In [16]:
accuracy_score(y_test,y_pred)

0.9065

In [17]:
bag.estimators_samples_[0].shape

(4000,)

In [18]:
bag.estimators_features_[0].shape

(10,)

## Bagging using SVM

In [19]:
bag = BaggingClassifier(
    estimator = SVC(),
    n_estimators =500,
    max_samples = 0.25,
    bootstrap = True,
    random_state = 42
)

In [20]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Bagging using SVM",accuracy_score(y_test,y_pred))

Bagging using SVM 0.925


## PASTING

In [22]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose = 1,
    n_jobs=-1
)

In [23]:

bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(y_test,y_pred))
     

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    5.2s remaining:   15.7s


Pasting classifier 0.9325


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    5.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


## Random Subspaces

In [25]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [26]:


bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))

Random Subspaces classifier 0.9275


In [27]:

bag.estimators_samples_[0].shape

(8000,)

In [28]:


bag.estimators_features_[0].shape

(5,)

## Random Patches

In [30]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)


In [31]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))

Random Patches classifier 0.927


## OOB Score

In [33]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

In [34]:


bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [35]:
bag.oob_score_

0.933

In [36]:
y_pred = bag.predict(X_test)

In [37]:
print("Accuracy",accuracy_score(y_test,y_pred))


Accuracy 0.93


## Bagging Tips

1. Bagging generally gives better results than Pasting                  
2. Good results come around the 25% to 50% row sampling mark                                 
3. Random patches and subspaces should be used while dealing with high dimensional data              
4. To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

## Applying GridSearchCV

In [38]:
from sklearn.model_selection import GridSearchCV

In [44]:
parameters = {
    'n_estimators': [50,100,500], 
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [45]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5)


In [46]:
search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=BaggingClassifier(),
             param_grid={'bootstrap': [True, False],
                         'max_features': [0.1, 0.4, 0.7, 1.0],
                         'max_samples': [0.1, 0.4, 0.7, 1.0],
                         'n_estimators': [50, 100, 500]})

In [47]:
search.best_params_
search.best_score_

0.935375

In [48]:
search.best_params_

{'bootstrap': True,
 'max_features': 0.7,
 'max_samples': 0.7,
 'n_estimators': 100}